In [4]:
import os
import warnings
import pickle    # chunk, vectorDB 저장한것 사용
from dotenv import load_dotenv

# 경고메세지 삭제
warnings.filterwarnings('ignore')
load_dotenv()

# openapi key 확인
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError('.env확인,  key없음')

# 필수 라이브러리 로드
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time
from pathlib import Path



class SimpleRAGSystem:
    '''간단한 RAG 시스템 래퍼 클래스'''
    def __init__(self, vectorstore, llm, retriever_k=3):
        self.vectorstore = vectorstore
        self.llm = llm
        self.retriever = vectorstore.as_retriever(seaarch_type = 'similarity', search_kwargs={'k':retriever_k})
        # self.retriever_chain = self._retriever_basic_chain()
        self.chain = self._build_chain()
    

    
    
    # def _retriever_basic_chain(self): # -------------------------> 내부 문서 찾는 프롬프트 수정 / 현재 코드 미완성
    #     '''retriever 검색'''
    #     basic_prompt = ChatPromptTemplate.from_messages([
    #             ("system", """당신은 제공된 문맥(Context)을 바탕으로 질문에 답변하는 AI 어시스턴트입니다.

    #         규칙:
    #         1. 제공된 문맥 내의 정보만을 사용하여 답변하세요.
    #         2. 문맥에 없는 정보는 "제공된 문서에서 해당 정보를 찾을 수 없습니다."라고 답하세요.
    #         3. 답변은 한국어로 명확하고 간결하게 작성하세요.
    #         4. 가능하면 구조화된 형태(목록, 번호 등)로 답변하세요."""),
    #             ("human", """문맥(Context): {context}

    #         질문: {question}

    #         답변:""")
    #         ])

    #     return (
            
    #     )


    def _build_chain(self): ### ----------------------------> 최종 사용자에게 전달되는 프롬프트 수정
        '''RAG 체인 구성''' 
        
        prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a professional AI research assistant specializing in HuggingFace Daily Papers.

        # Role
        Maintain context across multiple turns of conversation while answering based on retrieved papers.

        ## Conversational Guidelines 
        1. **Context Awareness**: Reference previous messages when relevant.

        2. **Consistency**: Maintain consistent terminology across the conversation.

        3. **Building Upon**: Build upon earlier answers with new retrieved information.

        4. **Clarification**: If asked for clarification, refer back to previously cited papers.

        ## Answer Rules
        1. **Source-based**: Only use information from the provided context.

        2. **Accuracy**: Do not make up information.

        3. **Language**: Always respond in the same language as the user's question.

        ## When Unable to Answer
        If you cannot find the information:
        "The user's question is not found in the most recent 5 weeks of data from Hugging Face. 
                 I will search using the GPT-4o model and provide an answer within 300 characters."""),
                
                ("human", """## Previous Conversation 
        {chat_history}

        ## Retrieved Papers 
        {context}

        ## Current Question 
        {question}

        ## Answer """)
            ])
        return(
            {
            'context': self.retriever | self._format_docs,
            'question': RunnablePassthrough(),
            'chat_history': lambda x: ""
            }
            | prompt
            | self.llm
            | StrOutputParser()
        )
    
    
    @staticmethod   
    def _format_docs(docs):
        return '\n\n'.join([doc.page_content for doc in docs])
    

    def ask(self, question:str) -> str:
        '''질문에 답변'''
        return self.chain.invoke(question)
    

    def ask_with_sources(self, question:str) -> dict : 
        '''질문에 답변 + 출처 반환'''
        answer = self.chain.invoke(question)
        sources = self.retriever.invoke(question)
        return {
            'answer' : answer
            # 'source' : [ doc.metadata.get('source', 'unknown') for doc in sources]
        }
    


if __name__ == '__main__' :
    
    # chunk 파일로 임시 확인
    def get_project_root():
        curr = Path().resolve()
        for parent in [curr] + list(curr.parents):
            if (parent / ".git").exists():
                return parent
        raise FileNotFoundError("프로젝트 루트 찾기 실패")

    PROJECT_ROOT = get_project_root()
    DATA_DIR = PROJECT_ROOT / "01_data/chunks"

    chunks_path = DATA_DIR / "chunks_all.pkl"

    with open(chunks_path, "rb") as f:
        chunks = pickle.load(f)

        
        # ---- metadata 제거 ----
        for doc in chunks:
            doc.metadata = {}

        vectorstore = Chroma.from_documents(
            documents=chunks,
            collection_name='test',
            embedding=OpenAIEmbeddings(model='text-embedding-3-small')
        )
    
        llm = ChatOpenAI( model = 'gpt-4o-mini', temperature=0 )

        rag_system = SimpleRAGSystem(vectorstore, llm)

        user_question = "META에서 만든 SAM 모델에 대해 설명해줘"
        result = rag_system.ask_with_sources(user_question)
        print(f'질문: {user_question}')
        print(f"답변: {result['answer']}")  
        # print(f"출처: {result['source']}")

질문: META에서 만든 SAM 모델에 대해 설명해줘
답변: Segment Anything Model (SAM) 3는 META에서 개발한 통합 모델로, 객체를 감지하고, 분할하며, 추적하는 기능을 제공합니다. 이 모델은 다양한 비전 작업을 수행할 수 있도록 설계되어 있으며, 특히 이미지 내에서 객체를 효과적으로 식별하고 처리하는 데 강점을 가지고 있습니다.
